# NanoPyx "Codeless" Jupyter Notebook

This notebook allows you to perform drift correction of 2D timelapses

To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step.
When running this notebook on Colab, it will automatically prompt you to connect to your google Drive, from where you can select the input data.
To save the output of this notebook, make sure to select the "save output" option. Output will be saved on the same directory as the loaded image. If using an example dataset it will be saved on the path Python is currently running (if locally running the notebook) or on Colab files (if running on Colab, files button on the left bar).
If you found this work useful for your research please consider citing our [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1)  and out Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398)

**Drift Correction** Romain F Laine et al 2019 J. Phys. D: Appl. Phys. 52 163001




In [ ]:
#@title Fix numpy version in Google Colab, can be skipped if running in local environment. Session will now restart automatically. You can then proceed to the next cell.
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q numpy==1.26.4
    !pip install -q mako==1.3.0

    print("Session will now restart automatically. You can then proceed to the next cell.")
    import os
    os.kill(os.getpid(), 9)




In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q "nanopyx[colab]"
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')
else:
    !pip install -q "nanopyx[jupyter]"

import io
import os
import cv2 as cv
import skimage
import nanopyx
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

try:
    from ezinput import EZInput as EasyGui
except ImportError:
    from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]




In [ ]:
#@title Load image stack
# Create a GUI
gui_data = EasyGui("Data Loader")
global own_data
own_data = True

def on_button_select_own(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_file_upload("upload")
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data_own", description="Load data")
    gui_data["load_data_own"].on_click(on_button_load_data_clicked)
    gui_data.show()

def on_button_select_example(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_dropdown("data_source", options=image_files,
                    value="Example dataset: "+example_datasets[4], remember_value=True)
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data", description="Load data")
    gui_data["load_data"].on_click(on_button_load_data_clicked_example)
    gui_data.show()

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = True
    # disable button
    gui_data["load_data_own"].disabled = True
    gui_data["load_data_own"].description = "Loading..."
    dataset_original = tiff.imread(gui_data["upload"].selected)
    gui_data["load_data_own"].disabled = False
    gui_data["load_data_own"].description = "Load data"
    gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, colormap=gui_data["cmaps"].value, continuous_update=True),)

def on_button_load_data_clicked_example(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = False
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."

    if gui_data["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)
    else:
        dataset_original = skimage.io.imread(gui_data["data_source"].value)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)

    # enable button
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    gui_data.save_settings()

gui_data.add_button("use_own_data", description="Use Own data")
gui_data["use_own_data"].on_click(on_button_select_own)
gui_data.add_button("use_example_data", description="Use Example data")
gui_data["use_example_data"].on_click(on_button_select_example)
gui_data.show()




# Drift Correction Parameters:  

Requires an image stack with shape: (time, rows, columns)  

- **Reference Frame:** Which frame to be used as reference. Either always use the first frame (better for fixed cells) or the previous frame (better for live cells).  
- **Max Expected Drift:** Maximum amount of expected drift in pixels.  
- **Time Averaging:** Whether to register each individual frame, if using 1, or how many frames to average before calculating drift correction (better for single molecule data). Output keeps the original number of frames, single frame drift estimation is calculated by interpolating using the calculated drift of the averaged image stack.




In [ ]:
# @title Create drift correction GUI
from nanopyx.methods.drift_alignment.estimator import DriftEstimator

gui_drift = EasyGui("Drift Correction")


def on_button_align(b):
    clear_output()
    gui_drift.show()
    if gui_drift["ref"].value == "First frame":
        ref_option = 0
    else:
        ref_option = 1
    avg = gui_drift["time_averaging"].value
    max_drift = gui_drift["max"].value
    global dataset_aligned
    gui_drift["align"].disabled = True
    gui_drift["align"].description = "Aligning..."

    estimator = DriftEstimator()
    dataset_aligned = estimator.estimate(
        dataset_original,
        time_averaging=avg,
        max_expected_drift=max_drift,
        ref_option=ref_option,
        apply=True,
    )

    if gui_drift["save"].value:
        if own_data:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            drift_table_path = path + os.sep + name + "_drift_table.csv"
            save_path = path + os.sep + name + "_aligned.tif"
        else:
            path = ""
            name = gui_data["data_source"].value.replace(
                "Example dataset: ", ""
            )
            drift_table_path = path + os.sep + name + "_drift_table.csv"
            save_path = name + "_aligned.tif"
        tiff.imwrite(save_path, dataset_aligned)
        txt = ""
        for key in estimator.estimator_table.params.keys():
            txt += (
                key + ";" + str(estimator.estimator_table.params[key]) + "\n"
            )
        txt += "Drift Table\n"
        txt += "XY;X;Y\n"
        for i in range(estimator.estimator_table.drift_table.shape[0]):
            txt += (
                str(estimator.estimator_table.drift_table[i][0])
                + ";"
                + str(estimator.estimator_table.drift_table[i][1])
                + ";"
                + str(estimator.estimator_table.drift_table[i][2])
                + "\n"
            )
        open(drift_table_path, "w").writelines(txt)
    gui_drift["align"].disabled = False
    gui_drift["align"].description = "Align"
    gui_drift._main_display.children = gui_drift._main_display.children + (
        stackview.slice(
            dataset_aligned,
            colormap=gui_drift["cmaps"].value,
            continuous_update=True,
        ),
    )


gui_drift.add_label(value="Drift Correction parameters:")
gui_drift.add_dropdown("ref", description="Reference frame", options=["First frame", "Previous frame"], value="First frame")
gui_drift.add_int_slider("max", description="Max expected drift", min=0, max=1000, value=10)
gui_drift.add_int_slider("time_averaging", description="Time averaging", min=1, max=dataset_original.shape[0], value=1)
gui_drift.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_drift.add_checkbox("save", description="Save Output", value=True)
gui_drift.add_button("align", description="Align")
gui_drift["align"].on_click(on_button_align)
gui_drift.show()




## Use the following cell only if you have a previously calculated drift table



In [ ]:
#@title Create apply drift correction GUI
import os
import sys

NANOPYX_INSTALLED = "nanopyx" in sys.modules
if not NANOPYX_INSTALLED:
    IN_COLAB = "google.colab" in sys.modules
    if IN_COLAB:
        !pip install -q "nanopyx[colab]"
        from google.colab import output
        output.enable_custom_widget_manager()
        from google.colab import drive
        drive.mount('/content/drive')
    else:
        !pip install -q "nanopyx[jupyter]"

import stackview
import tifffile as tiff
import matplotlib as mpl
from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.methods.drift_alignment.corrector import DriftCorrector
from IPython.display import display, clear_output

gui_drift_apply = EasyGui("Apply Drift Correction")

def on_button_apply(b):
    clear_output()
    gui_drift_apply.show()

    gui_drift_apply["Align image"].disabled = True
    gui_drift_apply["Align image"].description = "Aligning..."

    drift_table_path = gui_drift_apply["upload"].selected
    img_path = gui_drift_apply["upload image"].selected

    corrector = DriftCorrector()
    corrector.load_estimator_table(drift_table_path)
    aligned_image = corrector.apply_correction(tiff.imread(img_path))

    if gui_drift_apply["save"].value:
        path = gui_drift_apply["upload image"].selected_path
        name = gui_drift_apply["upload image"].selected_filename.split(".")[0]
        save_path = path + os.sep + name + "_aligned.tif"
        tiff.imwrite(save_path, aligned_image)



    gui_drift_apply["Align image"].disabled = False
    gui_drift_apply["Align image"].description = "Align"
    gui_drift_apply._main_display.children = gui_drift_apply._main_display.children + (stackview.slice(aligned_image, colormap=gui_drift_apply["cmaps"].value, continuous_update=True),)

gui_drift_apply.add_label(value="Load drift table:")
gui_drift_apply.add_file_upload("upload")
gui_drift_apply.add_label(value="Load image to align:")
gui_drift_apply.add_file_upload("upload image")
gui_drift_apply.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_drift_apply.add_checkbox("save", description="Save Output", value=True)
gui_drift_apply.add_button("Align image", description="Align image")
gui_drift_apply["Align image"].on_click(on_button_apply)
gui_drift_apply.show()

